In [1]:
import pandas as pd
import numpy as np

## Loading Dataset

In [2]:
frame = pd.read_csv('barterdata.csv')

In [3]:
frame.head()

,Title Name,View Count,Like Count,Tags,Category
0,Visual identity. Branding.,6845,933,"Logo', 'Package', 'Motion Graphics', 'Product'...",Environmental Design
1,Thomas Edisons Invention Factory,5997,224,"Comic Book', 'Character', 'Poster', 'Illustrat...",Logo Design
2,Hellorep.ai onboarding experience,1510,1080,"Web Design', 'Character', 'Magazine', 'Motion ...",Vehicle Wrap Design
3,3D Composition,739,930,"Illustration', 'Package', 'Character', 'Magazi...",Brand Identity Design
4,3D Composition,4647,384,"Comic Book', 'Typography', 'Web Design', 'Maga...",Illustration


In [4]:
frame.isnull().sum()

Title Name    0
View Count    0
Like Count    0
Tags          0
Category      0
dtype: int64

# # Changing all the tags into small letters

In [5]:
frame['Tags']=frame['Tags'].str.lower().str.replace("'","").str.replace(" ","").str.replace(","," ")

In [6]:
frame['Category']=frame['Category'].str.replace(" ","").str.lower()

In [7]:
frame.head()

,Title Name,View Count,Like Count,Tags,Category
0,Visual identity. Branding.,6845,933,logo package motiongraphics product character,environmentaldesign
1,Thomas Edisons Invention Factory,5997,224,comicbook character poster illustration brandi...,logodesign
2,Hellorep.ai onboarding experience,1510,1080,webdesign character magazine motiongraphics il...,vehiclewrapdesign
3,3D Composition,739,930,illustration package character magazine brandi...,brandidentitydesign
4,3D Composition,4647,384,comicbook typography webdesign magazine,illustration


# # Adding new column(final tag) in the Present Dataframe

In [8]:
frame['final_tag'] = frame['Tags'] + frame['Category']

In [9]:
frame.head()

,Title Name,View Count,Like Count,Tags,Category,final_tag
0,Visual identity. Branding.,6845,933,logo package motiongraphics product character,environmentaldesign,logo package motiongraphics product charactere...
1,Thomas Edisons Invention Factory,5997,224,comicbook character poster illustration brandi...,logodesign,comicbook character poster illustration brandi...
2,Hellorep.ai onboarding experience,1510,1080,webdesign character magazine motiongraphics il...,vehiclewrapdesign,webdesign character magazine motiongraphics il...
3,3D Composition,739,930,illustration package character magazine brandi...,brandidentitydesign,illustration package character magazine brandi...
4,3D Composition,4647,384,comicbook typography webdesign magazine,illustration,comicbook typography webdesign magazineillustr...


# # New Dataframe for the final model 

In [10]:
new_frame = frame[['Title Name','final_tag']]
new_frame.rename(columns = {"Title Name":"title"},inplace=True)

C:\Users\sonug\AppData\Local\Temp\ipykernel_9096\3050990701.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_frame.rename(columns = {"Title Name":"title"},inplace=True)


# # Applying steming in the final Dataframe(final tag column)

In [11]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [12]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [13]:
new_frame['final_tag'] = new_frame['final_tag'].apply(stem)

C:\Users\sonug\AppData\Local\Temp\ipykernel_9096\967637850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_frame['final_tag'] = new_frame['final_tag'].apply(stem)


In [14]:
new_frame.head(10)

,title,final_tag
0,Visual identity. Branding.,logo packag motiongraph product characterenvir...
1,Thomas Edisons Invention Factory,comicbook charact poster illustr brand socialm...
2,Hellorep.ai onboarding experience,webdesign charact magazin motiongraph illustr ...
3,3D Composition,illustr packag charact magazin brand motiongra...
4,3D Composition,comicbook typographi webdesign magazineillustr
5,Medical service - Mobile app,logo ui/ux motiongraph postermotiongraph
6,Blent | Case Study,magazin charact logo postermotiongraph
7,Visual identity. Branding.,poster webdesign magazinesignagedesign
8,Thomas Edisons Invention Factory,charact packag illustr webdesignenvironmentald...
9,Blent | Case Study,poster ui/ux packageenvironmentaldesign


# # Converting the final tag into Tfidf vector

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfdif = TfidfVectorizer(stop_words='english')

In [17]:
tfdif_new = tfdif.fit_transform(new_frame['final_tag'])

In [18]:
tfdif_new

<10000x242 sparse matrix of type '<class 'numpy.float64'>'
	with 54004 stored elements in Compressed Sparse Row format>

# #Cosine similarity on tfidf verctor

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
similarity = cosine_similarity(tfdif_new,tfdif_new)

In [21]:
indices = pd.Series(new_frame.index)

# # Final recommendation model

In [22]:
def recommend(title,cosine_sim = similarity):
    index = new_frame[new_frame["title"]==title].index[0]
    similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending=False)
    Top_5_post = list(similarity_scores.iloc[1:6].index)
    post = [list(new_frame.index)[i] for i in Top_5_post]
    post_titles = [new_frame.loc[i, "title"] for i in post]
    return list(zip( post_titles))

# Dumping the model

In [23]:
import pickle
pickle.dump(recommend,open('model_tfidf.pkl','wb'))